1. Import Libraries

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset, load_metric
import datasets
import torch
from bert_score import score

/home/arnav/miniconda3/envs/DLA2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Load Datasets

In [2]:
val_data = datasets.load_dataset("wmt16","de-en", split="validation")
test_data = datasets.load_dataset("wmt16","de-en", split="test")

print(len(val_data))
print(len(test_data))

2169
2999


- Load the T5 Model

In [3]:
# Load the T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Function to translate a sentence from German to English
def translate_sentence(sentence, model, tokenizer):
    input_text = f"translate English to German: {sentence}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(input_ids)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

- Validation Set Experiments

In [5]:
# iterate over the validation data and translate the English sentences to German
val_references = []
val_translations = []

i = 0
for item in val_data:
    i += 1
    english_sentence = item["translation"]["en"]
    german_sentence = item["translation"]["de"]

    translated_sentence = translate_sentence(english_sentence, model, tokenizer)

    val_references.append(german_sentence)
    val_translations.append(translated_sentence)
    print(f"Translated {i} sentences")


/home/arnav/miniconda3/envs/DLA2/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated 1 sentences
Translated 2 sentences
Translated 3 sentences
Translated 4 sentences
Translated 5 sentences
Translated 6 sentences
Translated 7 sentences
Translated 8 sentences
Translated 9 sentences
Translated 10 sentences
Translated 11 sentences
Translated 12 sentences
Translated 13 sentences
Translated 14 sentences
Translated 15 sentences
Translated 16 sentences
Translated 17 sentences
Translated 18 sentences
Translated 19 sentences
Translated 20 sentences
Translated 21 sentences
Translated 22 sentences
Translated 23 sentences
Translated 24 sentences
Translated 25 sentences
Translated 26 sentences
Translated 27 sentences
Translated 28 sentences
Translated 29 sentences
Translated 30 sentences
Translated 31 sentences
Translated 32 sentences
Translated 33 sentences
Translated 34 sentences
Translated 35 sentences
Translated 36 sentences
Translated 37 sentences
Translated 38 sentences
Translated 39 sentences
Translated 40 sentences
Translated 41 sentences
Translated 42 sentences
T

In [6]:
print(val_references[:5])
print(val_translations[:5])

['Die Premierminister Indiens und Japans trafen sich in Tokio.', 'Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Toko, um wirtschaftliche und sicherheitspolitische Beziehungen zu besprechen.', 'Herr Modi befindet sich auf einer fünftägigen Reise nach Japan, um die wirtschaftlichen Beziehungen mit der drittgrößten Wirtschaftsnation der Welt zu festigen.', 'Pläne für eine stärkere kerntechnische Zusammenarbeit stehen ganz oben auf der Tagesordnung.', 'Berichten zufolge hofft Indien darüber hinaus auf einen Vertrag zur Verteidigungszusammenarbeit zwischen den beiden Nationen.']
['In Tokio treffen sich die Premierminister Indiens und Japans', 'Der neue indische Premierminister Narendra Modi trifft sein japanisches Am', 'Herr Modi ist auf einer fünftägigen Reise nach Japan, um die wirtschaftliche', 'Auf der Tagesordnung stehen Pläne für eine verstärkte nukleare Zusammenar

In [7]:
from torchtext.data.utils import get_tokenizer

tokenizer_german = get_tokenizer("spacy", language="de_core_news_sm")
tokenizer_english = get_tokenizer("spacy", language="en_core_web_sm")

In [8]:
# Convert targets to the expected format for METEOR (list of lists, one reference per prediction)
meteor_targets = [[target] for target in val_references]

# Load METEOR and BLEU metrics
meteor_metric = load_metric('meteor', trust_remote_code=True)
bleu_metric = load_metric('bleu', trust_remote_code=True)

# Calculate METEOR score
meteor_results = meteor_metric.compute(predictions=val_translations, references=meteor_targets)

# BLEU expects tokenized references and predictions, and references must be in a list of lists of lists
bleu_targets_tokenized = [[[word for word in tokenizer_german(target)] for target in [ref]] for ref in val_references]
bleu_predictions_tokenized = [[word for word in tokenizer_german(pred)] for pred in val_translations]

# Calculate BLEU score
bleu_results = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized)

# Calculate BERTScore for German
P, R, F1 = score(val_translations, val_references, lang="de", verbose=True)

print("VAL DATA: ")
print("METEOR Score:", meteor_results["meteor"])
print("BLEU Score:", bleu_results["bleu"])
print(f"BERTScore-F1: {F1.mean()}")
print(f"BERTScore-Precision: {P.mean()}")
print(f"BERTScore-Recall: {R.mean()}")

/tmp/ipykernel_1123415/3885531215.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor_metric = load_metric('meteor', trust_remote_code=True)
[nltk_data] Downloading package wordnet to /home/arnav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/arnav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/arnav/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


calculating scores...
computing bert embedding.


100%|██████████| 68/68 [00:03<00:00, 21.58it/s]


computing greedy matching.


100%|██████████| 34/34 [00:00<00:00, 134.70it/s]

done in 3.41 seconds, 635.44 sentences/sec
VAL DATA: 
METEOR Score: 0.2681160952952596
BLEU Score: 0.12689119562034318
BERTScore-F1: 0.7942118644714355
BERTScore-Precision: 0.8268641829490662
BERTScore-Recall: 0.7662411332130432


In [9]:
# Iterate over the test data and translate the German sentences to English
test_references = []
test_translations = []

i = 0
for item in test_data:
    i += 1
    english_sentence = item["translation"]["en"]
    german_sentence = item["translation"]["de"]

    translated_sentence = translate_sentence(english_sentence, model, tokenizer)

    test_references.append(german_sentence)
    test_translations.append(translated_sentence)

    print(f"Translated {i} sentences")

/home/arnav/miniconda3/envs/DLA2/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated 1 sentences
Translated 2 sentences
Translated 3 sentences
Translated 4 sentences
Translated 5 sentences
Translated 6 sentences
Translated 7 sentences
Translated 8 sentences
Translated 9 sentences
Translated 10 sentences
Translated 11 sentences
Translated 12 sentences
Translated 13 sentences
Translated 14 sentences
Translated 15 sentences
Translated 16 sentences
Translated 17 sentences
Translated 18 sentences
Translated 19 sentences
Translated 20 sentences
Translated 21 sentences
Translated 22 sentences
Translated 23 sentences
Translated 24 sentences
Translated 25 sentences
Translated 26 sentences
Translated 27 sentences
Translated 28 sentences
Translated 29 sentences
Translated 30 sentences
Translated 31 sentences
Translated 32 sentences
Translated 33 sentences
Translated 34 sentences
Translated 35 sentences
Translated 36 sentences
Translated 37 sentences
Translated 38 sentences
Translated 39 sentences
Translated 40 sentences
Translated 41 sentences
Translated 42 sentences
T

In [10]:
# Convert targets to the expected format for METEOR (list of lists, one reference per prediction)
meteor_targets = [[target] for target in test_references]

# Calculate METEOR score
meteor_results = meteor_metric.compute(predictions=test_translations, references=meteor_targets)

# BLEU expects tokenized references and predictions, and references must be in a list of lists of lists
bleu_targets_tokenized = [[[word for word in tokenizer_german(target)] for target in [ref]] for ref in test_references]
bleu_predictions_tokenized = [[word for word in tokenizer_german(pred)] for pred in test_translations]

# Calculate BLEU score
bleu_results = bleu_metric.compute(predictions=bleu_predictions_tokenized, references=bleu_targets_tokenized)

# Calculate BERTScore for German
P, R, F1 = score(test_translations, test_references, lang="de", verbose=True)

print("TEST DATA: ")
print("METEOR Score:", meteor_results["meteor"])
print("BLEU Score:", bleu_results["bleu"])
print(f"BERTScore-F1: {F1.mean()}")
print(f"BERTScore-Precision: {P.mean()}")
print(f"BERTScore-Recall: {R.mean()}")

calculating scores...
computing bert embedding.


100%|██████████| 93/93 [00:04<00:00, 22.11it/s]


computing greedy matching.


100%|██████████| 47/47 [00:00<00:00, 160.34it/s]


done in 4.52 seconds, 664.21 sentences/sec
TEST DATA: 
METEOR Score: 0.2879851395419705
BLEU Score: 0.13723357750231824
BERTScore-F1: 0.8000087141990662
BERTScore-Precision: 0.8345420956611633
BERTScore-Recall: 0.7703850269317627
